In [18]:
import numpy as np

from bokeh import mpl
import bokeh.plotting as blt
import bokeh.models as bm
blt.output_notebook()

import ipywidgets as iw
from IPython.display import display
from IPython.display import clear_output

import os
import re
import json
import glob
import itertools as it
from contextlib import contextmanager
import collections as cl
import os.path as pt

Graph = cl.namedtuple('Graph', ['poses', 'odom', 'loop'])

Loading BokehJS ...

In [29]:
def draw_connections(fig, idx_pairs_list, poses_mat, colors):
    idx_pairs_mat = np.matrix(idx_pairs_list)
    start_pts = poses_mat[idx_pairs_mat[:,0].A1.tolist(), :]
    end_pts = poses_mat[idx_pairs_mat[:,1].A1.tolist(), :]
    fig.segment(start_pts[:,0].A1.tolist(), start_pts[:,1].A1.tolist(), 
                end_pts[:,0].A1.tolist(), end_pts[:,1].A1.tolist(), color=colors)

    
def colorstr(r, g, b):
    return "#%02x%02x%02x" % (r, g, b)
    
    
def interp(c1, c2, w):
    return tuple([b * w + (a * (1 - w)) for a, b in zip(c1, c2)])


def draw_switch_loop_closures(fig, conn_lists, poses_mat): 
    if not len(conn_lists):
        return
    idx_pairs = [c[0:2] for c in conn_lists]
    weights = [c[2] for c in conn_lists]
    good_clr = (255, 179, 122)
    bad_clr = (200, 200, 200)
    
    colors = [colorstr(*interp(bad_clr, good_clr, w)) for w in weights]
    draw_connections(fig, idx_pairs, poses_mat, colors)

    
def draw_poses(fig, poses):    
    fig.scatter(poses[:,0].A1.tolist(), poses[:,1].A1.tolist(), color=colorstr(0, 204, 255), radius=.1)
    
    
@contextmanager    
def bokeh_fig(x_range, y_range):
    f = blt.figure(plot_width=750, plot_height=750, x_range=x_range, y_range=y_range)
    yield f
    blt.show(f)
    

def draw_graph(g):
    poses = np.matrix(g.poses)[:,1:]
    mx = np.max(poses[:, 0:2])
    mn = np.min(poses[:, 0:2])
    x_range = (mn, mx)
    y_range = (mn, mx)
    with bokeh_fig(x_range, y_range) as f:
        draw_connections(f, g.odom, poses, colorstr(0, 204, 255))
        draw_switch_loop_closures(f, g.loop, poses)
        draw_poses(f, poses)
    switch_constraint_fig(g.loop)
    
        
def switch_constraint_fig(loops):
    f = blt.figure(plot_width=200, plot_height=200, x_range=[0, 1], y_range=[0, 1])
    switch_vars = [cons[2] for cons in loops]
    hist, edges = np.histogram(switch_vars, bins=10, range=[0.,1.])
    hist = hist / float(np.sum(hist))
    f.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
           fill_color="#036564", line_color="#033649")
    blt.show(f)
    print hist
    
    
def open_graph(fname):
    with open(fname, 'r') as f:
        obj = json.load(f)
        print("poses ", len(obj['poses']))
        print("loops ", len(obj['loop']))
        print("odom  ", len(obj['odom']))    
        obj['poses'].sort(key=lambda x: x[0])
        return Graph(**obj)
        

def process_vertex(g, tokens):
    v = [int(tokens[1]), 
         float(tokens[2]), 
         float(tokens[3]), 
         float(tokens[4])]
    
    return g._replace(poses=g.poses + [v])
    
    
def process_edge(g, tokens):
    c = [int(tokens[1]), int(tokens[2])]
    pose = [float(tokens[3]), float(tokens[4]), float(tokens[5])]
    c.sort()
    if c[1] == c[0] + 1:
        return g._replace(odom=g.odom + [c + pose])
    else:
        return g._replace(loop=g.loop + [c + [1.] + pose])


def process_line(g, line):    
    tokens = line.split()
    if (tokens[0] == 'VERTEX_SE2'):
        return process_vertex(g, tokens)
    
    elif (tokens[0] == 'EDGE_SE2'):
        return process_edge(g, tokens)

    
def load_g2o(fname):
    with open(fname, 'r') as fi:
        return reduce(process_line, iter(fi), Graph([],[],[]))

    
def test_process_edge():
    l = "EDGE_SE2 279 288 -0.984381 -0.0110328 -3.11799 44.72135955 0 0 44.72135955 0 44.72135955"
    print process_edge(Graph([],[],[]), l.split())
    
    
def test_process_vertex():
    l = "VERTEX_SE2   0 0 0 0"
    print process_vertex(Graph([],[],[]), l.split())
    
    
def test_process_line():
    ll = ["EDGE_SE2 279 288 -0.984381 -0.0110328 -3.11799 44.72135955 0 0 44.72135955 0 44.72135955",
         "VERTEX_SE2   0 0 0 0"]
    print reduce(process_line, ll, Graph([],[],[]))

    
def test_load_g2o():
    fname = "../../datasets/manhattan/originalDataset/g2o/manhattanOlson3500.g2o"
    g = load_g2o(fname)
    for p in g.poses[0:5]:
        print p


def list_files(path):      
    for root, dirs, files in os.walk(path):
        for f in files:
            if f.endswith(".g2o"):
                yield os.path.join(root, f)
                
test_process_edge()
test_process_vertex()
test_process_line()
test_load_g2o()
        

Graph(poses=[], odom=[], loop=[[279, 288, 1.0, -0.984381, -0.0110328, -3.11799]])
Graph(poses=[[0, 0.0, 0.0, 0.0]], odom=[], loop=[])
Graph(poses=[[0, 0.0, 0.0, 0.0]], odom=[], loop=[[279, 288, 1.0, -0.984381, -0.0110328, -3.11799]])
[0, 0.0, 0.0, 0.0]
[1, 1.03039, 0.0113498, -0.0129577]
[2, 2.04345, -0.060422, -0.0261827]
[3, 3.07055, -0.094779, -0.0213499]
[4, 3.07998, 0.909609, 1.54544]


In [50]:
def draw_json_files(prefix, value=125): 
    def draw(idx):
        fname = pt.join(prefix[0], "myfile{}.json".format(idx))
        print("drawing " + fname)
        draw_graph(open_graph(fname))
        
    files = glob.glob(pt.join(prefix[0], "*.json"))
    nfiles = max([int(re.match("({}myfile)(\d*)(\.json)".format(prefix[0]), n).groups()[1]) for n in files]) + 1      
    s = iw.IntSlider(min=1, max=nfiles - 1, step=1, value=value) 
    w = iw.interactive(draw, idx=s)
    display(w)
    
draw_json_files(["logs/intel_switchable/"])    

drawing logs/intel_switchable/myfile129.json
('poses ', 130)
('loops ', 12)
('odom  ', 129)


[ 0.5  0.   0.   0.   0.   0.   0.   0.   0.   0.5]


In [38]:
draw_json_files(["logs/intel2/"])    

drawing logs/intel2/myfile134.json
('poses ', 135)
('loops ', 13)
('odom  ', 134)


[ 0.          0.          0.          0.          0.          0.
  0.07692308  0.07692308  0.38461538  0.46153846]


In [61]:
draw_json_files(["logs/intel_switchable_half_info_mat/"], value=185)    

drawing logs/intel_switchable_half_info_mat/myfile187.json
('poses ', 188)
('loops ', 68)
('odom  ', 187)


[ 0.17647059  0.01470588  0.02941176  0.01470588  0.02941176  0.05882353
  0.02941176  0.07352941  0.08823529  0.48529412]


In [62]:
draw_json_files(["logs/intel_switchable_half_info_mat2/"], value=185)    

drawing logs/intel_switchable_half_info_mat2/myfile188.json
('poses ', 189)
('loops ', 69)
('odom  ', 188)


[ 0.23188406  0.          0.01449275  0.01449275  0.04347826  0.01449275
  0.02898551  0.08695652  0.13043478  0.43478261]


In [67]:
draw_json_files(["logs/intel_switchable_half_info_mat3/"], value=187)    

drawing logs/intel_switchable_half_info_mat3/myfile187.json
('poses ', 188)
('loops ', 68)
('odom  ', 187)


[ 0.16176471  0.          0.01470588  0.          0.11764706  0.10294118
  0.05882353  0.07352941  0.05882353  0.41176471]


In [52]:
draw_json_files(["logs/manhattanOlson3500/"], value=3499)   

drawing logs/manhattanOlson3500/myfile3499.json
('poses ', 3500)
('loops ', 0)
('odom  ', 5598)


/home/hai/vcs/vertigo/examples/robustISAM2/isam_env/lib/python2.7/site-packages/ipykernel/__main__.py:57: RuntimeWarning: invalid value encountered in divide


[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan]


In [32]:
def draw_g2o():
    dsets = list(list_files("../../datasets"))
    dd = iw.Dropdown(options=dsets, value=dsets[0], description='Dataset:')
    display(dd)
    
    def cb(v):
        clear_output()
        print v['new']
        g = load_g2o(v['new'])   
        draw_graph(g)
        
    dd.observe(cb, names='value')

draw_g2o()

../../datasets/intel/originalDataset/intel.g2o


[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]


('poses ', 941)
('loops ', 886)
('odom  ', 940)


[ 0.00225734  0.00112867  0.00225734  0.00225734  0.00790068  0.02370203
  0.04288939  0.08803612  0.15575621  0.6738149 ]
